In [1]:
import pandas as pd

results = pd.read_csv('results/gsm8k100.csv')
results

,correct,compare_mode,gold_answer,pred_answer,question
0,False,numeric,18,"system\nYou are Qwen, created by Alibaba Cloud...",Janet’s ducks lay 16 eggs per day. She eats th...
1,True,numeric,3,"system\nYou are Qwen, created by Alibaba Cloud...",A robe takes 2 bolts of blue fiber and half th...
2,False,numeric,70000,"system\nYou are Qwen, created by Alibaba Cloud...",Josh decides to try flipping a house. He buys...
3,True,numeric,540,"system\nYou are Qwen, created by Alibaba Cloud...",James decides to run 3 sprints 3 times a week....
4,True,numeric,20,"system\nYou are Qwen, created by Alibaba Cloud...","Every day, Wendi feeds each of her chickens th..."
5,False,numeric,64,"system\nYou are Qwen, created by Alibaba Cloud...",Kylar went to the store to buy glasses for his...
6,False,numeric,260,"system\nYou are Qwen, created by Alibaba Cloud...",Toulouse has twice as many sheep as Charleston...
7,False,numeric,160,"system\nYou are Qwen, created by Alibaba Cloud...",Carla is downloading a 200 GB file. Normally s...
8,False,numeric,45,"system\nYou are Qwen, created by Alibaba Cloud...",John drives for 3 hours at a speed of 60 mph a...
9,False,numeric,460,"system\nYou are Qwen, created by Alibaba Cloud...",Eliza's rate per hour for the first 40 hours s...


In [ ]:
results.iloc[1]['pred_answer']

'system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.\nuser\nA robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?\nassistant\nTo determine the total number of bolts of fiber needed for the robe, we can break down the problem as follows:\n\n1. The robe requires 2 bolts of blue fiber.\n2. It also requires half that amount of white fiber. Since it needs 2 bolts of blue fiber, half of that would be \\( \\frac{2}{2} = 1 \\) bolt of white fiber.\n\nAdding these together:\n- Blue fiber: 2 bolts\n- White fiber: 1 bolt\n\nTotal bolts of fiber needed = 2 (blue) + 1 (white) = 3 bolts.\n\nSo, the robe takes a total of 3 bolts of fiber.'